# Initialize paths to data

In [1]:
train_path_dataset = '/kaggle/input/github/Assignment_3/data/train.jsonl'
test_path_dataset = '/kaggle/input/github/Assignment_3/data/test.jsonl'

# Install and import libraries

In [2]:
# Install required packages
!pip install -q peft transformers datasets evaluate seqeval wandb spancat thinc==8.2.3
!python3 -m spacy download ru_core_news_lg

ERROR: Could not find a version that satisfies the requirement spancat (from versions: none)
ERROR: No matching distribution found for spancat
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


# Preprocessing

In [32]:
import pandas as pd
import spacy
from spacy.training import Example
from typing import List
import random
from spacy.util import compounding, minibatch
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
from thinc.api import RAdam
from tqdm import tqdm

In [33]:
# Read training data from a JSONL file
json_data = pd.read_json(train_path_dataset, lines=True)
json_data = json_data[json_data.columns[::-1]]

# Display training data from a JSONL file
json_data

,id,sentences,ners
0,0,Бостон взорвали Тамерлан и Джохар Царнаевы из ...,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS..."
1,1,Умер избитый до комы гитарист и сооснователь г...,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],..."
2,2,Путин подписал распоряжение о выходе России из...,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O..."
3,3,Бенедикт XVI носил кардиостимулятор\nПапа Римс...,"[[0, 11, PERSON], [36, 47, PROFESSION], [49, 6..."
4,4,Обама назначит в Верховный суд латиноамериканк...,"[[0, 4, PERSON], [17, 29, ORGANIZATION], [48, ..."
...,...,...,...
514,514,Глава Малайзии: мы не хотим противостоять Кита...,"[[42, 46, COUNTRY], [82, 87, COUNTRY], [104, 1..."
515,515,«Союз» впервые пристыковался к МКС за 6 часов\...,"[[1, 4, PRODUCT], [31, 33, FACILITY], [35, 44,..."
516,516,Трамп и Путин сделали совместное заявление к 7...,"[[0, 4, PERSON], [8, 12, PERSON], [45, 52, AGE..."
517,517,Российский магнат устроил самую дорогую свадьб...,"[[0, 9, NATIONALITY], [58, 72, PERSON], [101, ..."


In [40]:
def get_span(doc, ners):
    """
    Get span entities.

    Args:
    - doc (str): text.
    - ners (List[List[int, int, str]]): List of dataset entities.

    Returns:
    - List[Tuple[int, int, str]]: Tuple containing span entities.
    """
    span_ents = []

    for start, end, tag in ners:
        span_ent = (start, end+1, label)
        annotation = {"spans": {span_key: [span_ent]}}
        try: # try to create SpanCat object
            Example.from_dict(doc, annotation)
        except: # in case of an error in dataset
            continue 
        span_ents.append(span_ent)
        
    return span_ents

In [41]:
span_ners = []
for i, row in tqdm(json_data.iterrows()):
    text = row['sentences']
    doc = nlp(text)
    span_ents = get_span(doc, row['ners'])
    span_ners.append(span_ents)
json_data['span_ners'] = span_ners

519it [02:15,  3.84it/s]


In [42]:
json_data.head(3)

,id,sentences,ners,span_ners
0,0,Бостон взорвали Тамерлан и Джохар Царнаевы из ...,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS...","[(0, 6, WORK_OF_ART), (16, 24, WORK_OF_ART), (..."
1,1,Умер избитый до комы гитарист и сооснователь г...,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],...","[(21, 29, WORK_OF_ART), (53, 68, WORK_OF_ART),..."
2,2,Путин подписал распоряжение о выходе России из...,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O...","[(0, 5, WORK_OF_ART), (37, 43, WORK_OF_ART), (..."


# Training

In [43]:
# Load pretrained spaCy model
nlp = spacy.load("ru_core_news_lg")

# Define span key
span_key = "sc"

In [44]:
# Create spaCy compliant training data
train_data = []
for i, row in tqdm(json_data.iterrows()):
    doc = nlp(row['sentences'])
    annotation = {"spans": {span_key: row['span_ners']}}
    train_data.append(Example.from_dict(doc, annotation))

519it [01:38,  5.28it/s]


In [46]:
# Define label list
label_list = [
    'AGE', 'AWARD', 'CITY', 'COUNTRY', 'CRIME', 'DATE', 'DISEASE', 'DISTRICT', 'EVENT', 'FACILITY',
    'FAMILY', 'IDEOLOGY', 'LANGUAGE', 'LAW', 'LOCATION', 'MONEY', 'NATIONALITY', 'NUMBER', 'ORDINAL',
    'ORGANIZATION', 'PENALTY', 'PERCENT', 'PERSON', 'PRODUCT', 'PROFESSION', 'RELIGION',
    'STATE_OR_PROVINCE', 'TIME', 'WORK_OF_ART'
]

# Define spancat config
config = {
    "threshold": 0.5,
    "spans_key": span_key,
    "max_positive": 1,
    "model": DEFAULT_SPANCAT_MODEL,
    "suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3]},
}

In [47]:
# Add spancat component to spaCy pipeline
nlp.add_pipe("spancat", config=config)
span = nlp.get_pipe('spancat')

# Add labels to spancat component
for label in label_list:
    span.add_label(label)

In [48]:
# Define optimizer
optimizer = RAdam(
    learn_rate=0.01,
    beta1=0.9,
    beta2=0.999,
    eps=1e-08,
    grad_clip=1.0,
    use_averages=True,
)

# Get pipes for training
pipe_exceptions = ["spancat"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Initialize spaCy object
nlp.initialize()

[2024-04-28 19:37:36,520] [INFO] Created vocabulary
[2024-04-28 19:37:36,521] [INFO] Finished initializing nlp object


In [49]:
# Start training the spancat component
all_losses = []
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(10):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(list(batch), losses=losses, drop=0.2, sgd=optimizer)
        print("epoch: {} Losses: {}".format(iteration, str(losses)))
        all_losses.append(losses['spancat'])

epoch: 0 Losses: {'spancat': 359412.05878448486}
epoch: 1 Losses: {'spancat': 12057.873386383057}
epoch: 2 Losses: {'spancat': 9336.083824157715}
epoch: 3 Losses: {'spancat': 8123.159854888916}
epoch: 4 Losses: {'spancat': 7097.439971923828}
epoch: 5 Losses: {'spancat': 6510.6135540008545}
epoch: 6 Losses: {'spancat': 6016.545631408691}
epoch: 7 Losses: {'spancat': 5636.657283782959}
epoch: 8 Losses: {'spancat': 5245.4140625}
epoch: 9 Losses: {'spancat': 4911.886821746826}


# Prediction

In [50]:
# Read test data from a JSONL file
test_data = pd.read_json(test_path_dataset, lines=True)
test_data = test_data[test_data.columns[::-1]]

In [52]:
from spacy import displacy

# Perform NER on test data
ners = []
for text in tqdm(test_data['senences']):
    doc = nlp(text)
    preds = []
    spans = doc.spans[span_key]
    ents_parse = displacy.parse_spans(doc, options={"spans_key": span_key})
    for span in ents_parse['spans']:
        preds.append([span['start'], span['end'], span['label']])

    ners.append(preds)

test_data['ners'] = ners

  0%|          | 0/65 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
100%|██████████| 65/65 [00:14<00:00,  4.54it/s]


In [53]:
# Save results to a JSONL file
submission_df = test_data[['id', 'ners']]
submission_df.to_json('test.jsonl', lines=True, orient='records', force_ascii=False)

In [54]:
submission_df

,id,ners
0,584,"[[0, 8, WORK_OF_ART], [10, 20, WORK_OF_ART], [..."
1,585,"[[0, 5, WORK_OF_ART], [18, 25, WORK_OF_ART], [..."
2,586,"[[0, 11, WORK_OF_ART], [39, 44, WORK_OF_ART], ..."
3,587,"[[2, 8, WORK_OF_ART], [17, 23, WORK_OF_ART], [..."
4,588,"[[73, 79, WORK_OF_ART], [108, 115, WORK_OF_ART..."
...,...,...
60,644,"[[0, 4, WORK_OF_ART], [13, 23, WORK_OF_ART], [..."
61,645,"[[63, 69, WORK_OF_ART], [118, 130, WORK_OF_ART..."
62,646,"[[30, 38, WORK_OF_ART], [91, 107, WORK_OF_ART]..."
63,647,"[[38, 41, WORK_OF_ART], [49, 56, WORK_OF_ART],..."
